# Index
1. ### [Get list of URLs to parse](#URLlist)
1. ### [Content scraping](#content)
1. ### [Saving to CSV file](#export)

In [1]:
possible_labels = ['true', 'mostly true', 'false', 'mostly false', 'miscaptioned', 'misleading', 'incorrect']

<a id='URLlist'></a>

## Get list of URLs to parse

In [13]:
with open('factcheckhub_fc_links.txt', 'r') as f:
    file_contents = f.read()
    fc_links = file_contents.split('\n')

In [14]:
len(fc_links)

145

In [15]:
halfway_point = len(fc_links) // 2
fc_links1 = fc_links[:halfway_point]
fc_links2 = fc_links[halfway_point:]

## fc_links1

In [18]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

def replace_blanks_and_linebreaks(s):
        s = s.replace('\xa0', ' ')
        s = s.replace('\n', ' ')
        s = s.replace('\t', ' ')
        s = s.replace('\xa0\n', ' ')
        s = re.sub(r'\s+', ' ', s)
        return s

claims, verdicts, justifications, dates, urls = [],[],[],[],[]

field_list1 = []
possible_labels = ['true', 'mostly true', 'false', 'mostly false', 'miscaptioned', 'misleading', 'incorrect']

#url = 'https://factcheckhub.com/anambradecides2021image-does-not-show-apc-chairman-collapsing-after-losing-lg-to-pdp/'
api_key = '11ceea2c-a709-484c-8e5d-48af1fe50f69'

for url in fc_links1:
    try:
        url_tuple = ()
        response = requests.get(url='https://proxy.scrapeops.io/v1/',
                            params={'api_key': api_key,
                                    'url': url,
                                    'render_js': True})
        src = response.content     
        soup = BeautifulSoup(src, 'html.parser')

        # # # # # # # # # # # # # # # # #
        # get all text from the article #
        # # # # # # # # # # # # # # # # #

        article_text = replace_blanks_and_linebreaks(soup.text).strip()



        # # # # # # # # #
        # extract claim #
        # # # # # # # # #

        pattern_claim = re.compile(r'.*CLAIM.*')
        claim_header = soup.find(text=pattern_claim)
        #claim_header = soup.find(text='THE CLAIM:')
        claim = claim_header.find_next('p').text
        #claims.append(claim)
        if len(claim) > 1:
            url_tuple = url_tuple + (claim,)
            print(f'Claim:\n{claim}\n')



        # # # # # # # # # # #
        # extract evidence  #
        # # # # # # # # # # #

        start_word = "FINDINGS"
        end_word = "VERDICT"
        pattern = re.compile(f"{start_word}(.*?){end_word}", re.DOTALL)
        evidence = pattern.findall(article_text)[0].split('THE ')[0].strip()
        if evidence.startswith(': '): evidence = evidence.split(': ')[1]
        #justifications.append(evidence)
        if len(evidence) > 1:
            url_tuple = url_tuple + (evidence,)
            print(f'Evidence:\n{evidence}\n')


        # # # # # # # # # #
        # extract verdict #
        # # # # # # # # # #

        pattern_verdict = re.compile(r'.*VERDICT.*')
        verdict_header = soup.find(text=pattern_verdict)
        #claim_header = soup.find(text='THE CLAIM:')
        verdict_par = verdict_header.find_next('p').text
        pattern_verdict_is = re.compile(r'\bis\b\s+(\w+)')
        match = pattern_verdict_is.search(verdict_par)
        verdict = match.group(1)
        #verdicts.append(verdict)
        if verdict.lower() in possible_labels and len(verdict) > 1:
            url_tuple = url_tuple + (verdict,)
            print(f'Verdict:\n{verdict}\n')



        # # # # # # # # #
        # extract date  #
        # # # # # # # # #

        date = str(soup.find('header', class_='entry-header').text.split('By')[-1].split('on')[-1]).strip()
        dt_object = datetime.strptime(date, '%B %d, %Y')
        formatted_date = dt_object.strftime('%d-%m-%Y')
        #dates.append(formatted_date)
        if len(formatted_date) > 1:
            url_tuple = url_tuple + (formatted_date,)
            print(f'Date:\n{formatted_date}\n')



        # # # # # # # # #
        #  extract url  #
        # # # # # # # # #
        
        #urls.append(url)
        if len(url) > 1:
            url_tuple = url_tuple + (url,)
            print(f'URL:\n{url}\n')

        
        if len(url_tuple) == 5:
            print(url_tuple)
            field_list1.append(url_tuple)
        
    except:
        print('Unwanted page\n')
        continue

Unwanted page

Unwanted page

Claim:
FINDINGS: As of 5pm on Tuesday (March 21), INEC is yet to declare the winner of the gubernatorial election in Enugu state. A press statement released by the commission says collation has been suspended in the state. Vanguard, Thisday live, and Tribune online all reported the suspension of the collation.

Evidence:
As of 5pm on Tuesday (March 21), INEC is yet to declare the winner of the gubernatorial election in Enugu state. A press statement released by the commission says collation has been suspended in the state. Vanguard, Thisday live, and Tribune online all reported the suspension of the collation.

Unwanted page

Claim:
FINDINGS: The NFC contacted Sam Emeka Okpala, the Lagos state Secretary of the Labour Party, who explained that the widely circulated letter was forged.

Evidence:
The NFC contacted Sam Emeka Okpala, the Lagos state Secretary of the Labour Party, who explained that the widely circulated letter was forged. Also, the LP Lagos sta

Unwanted page

Unwanted page

Claim:
Peter Obi claimed that over 130 million live in poverty in Nigeria.

Evidence:
According to a recent report by the National Bureau of Statistics (NBS) on multi-dimensional poverty, about 133 million people in the country, representing 63 per cent of the population, are living in different categories of poverty. The report stated that 65 per cent of the poor (86 million people) live in the North, while 35 per cent (nearly 47 million) live in the South. The National Multi-Dimensional Poverty Index (MPI) report is 0.257, indicating that poor people in Nigeria experience just over one-quarter of all possible deprivations. The report stresses that poverty levels across states vary significantly, with the incidence of multi-dimensional poverty ranging from a low of 27 per cent in Ondo to a high of 91 per cent in Sokoto. According to the report, over half of the population of Nigeria are multi-dimensionally poor and cook with dung, wood or charcoal, rather

Unwanted page

Unwanted page

Unwanted page

Unwanted page

Claim:
Video shows INEC REC brutalized in Adamawa state.

Evidence:
Findings by The FactCheckHub show that the claim is FALSE. While the context of the video is yet to be ascertain, The FactCheckHub found out that the man in the video is not the INEC Resident Electoral Commissioner (REC) in Adamawa State but the INEC national commissioner and chairman, board of the Electoral Institute, Abdullahi Abdul Zuru. INEC national commissioner and chairman, board of the Electoral Institute, Abdullahi Zuru. Abdullahi Abdul Zuru is a former vice-chancellor at Usmanu Danfodiyo University, Sokoto. However, the controversial INEC Resident Electoral Commissioner (REC) in Adamawa State is Hudu Ari. Hudu Ari, INEC Resident Electoral Commissioner for Adamawa state.

Verdict:
FALSE

Date:
16-04-2023

URL:
https://factcheckhub.com/video-does-not-show-inec-rec-for-adamawa-brutalized/

('Video shows INEC REC brutalized in Adamawa state.', 'Findings 

Claim:
US criticizes Nigeria’s 2023 presidential election, asks INEC to prove that the declared results are correct.

Evidence:
Findings by The FactCheckHub show that the claim is MISLEADING. Checks revealed that the video was a footage from the conference of National Democratic Coalition (NADECO) – a pro-democracy group – held at the National Press Club, Washington DC in the United States on March 8, 2023. The FactCheckHub found the complete video in which W. Bruce DelValle, the General Counsel of NADECO, criticized INEC for certain issues in the electoral processes that saw Tinubu emerged as Nigeria’s President-elect. However, the stance of the NADECO group doesn’t reflect the stance of the United States Government. The Cable reported that the US State Department had congratulated Tinubu. The Vanguard also reported same.

Verdict:
MISLEADING

Date:
09-04-2023

URL:
https://factcheckhub.com/no-us-government-did-not-criticize-nigerias-presidential-election/

('US criticizes Nigeria’s 2

Claim:
Photo shows Sanwo-Olu is now a dispatch rider.

Evidence:
Findings by The FactCheckHub show that the claim is FALSE. Extensive search led us to the original video which was uploaded on social media by an Instagram user, @asg.me on March 15, 2023. Further checks on the footage revealed that the video is a comedy skit. In the footage, the dispatch rider later removed his helmet which shows that it’s a different person and not Babajide Sanwo-Olu as claimed in several social media posts. The image was taken from the comedy skit.

Verdict:
FALSE

Date:
16-03-2023

URL:
https://factcheckhub.com/lagos-gubernatorial-doctored-images-of-sanwo-olu-circulate-online/

('Photo shows Sanwo-Olu is now a dispatch rider.', 'Findings by The FactCheckHub show that the claim is FALSE. Extensive search led us to the original video which was uploaded on social media by an Instagram user, @asg.me on March 15, 2023. Further checks on the footage revealed that the video is a comedy skit. In the footage, 

...

Claim:
THE FINDINGS: The Facebook post made on 26 February 2023 provided no evidence. The report differs from the headline. It says Abubakar was “projected to become the first candidate to meet the mandatory 25% requirement of votes cast in at least 24 States”. However, as of 7a.m. on Monday, February 27, 2023, INEC had declared presidential election results from less than 20 states.

Evidence:
The Facebook post made on 26 February 2023 provided no evidence. The report differs from the headline. It says Abubakar was “projected to become the first candidate to meet the mandatory 25% requirement of votes cast in at least 24 States”. However, as of 7a.m. on Monday, February 27, 2023, INEC had declared presidential election results from less than 20 states.

Unwanted page

Claim:
THE FINDINGS: From the comment section, the NFC observed that some social media users were curious about which Oluomo was actually killed. Although the report states that PDP supporter Akinlabi Akinnaso, also know

Claim:
Tinubu claimed that Atiku Abubakar was a Senate President.

Evidence:
Findings by The FactCheckHub show that the claim is FALSE. Atiku Abubakar, one of the front-runners in the 2023 presidential election, is a former Nigerian Vice President during the democratic regime of Chief Olusegun Obasanjo (1999 – 2007) and a retired Officer of the Nigerian Customs Service (NCS). He ran as governor of Adamawa State four times before becoming Obasanjo’s running mate in 1999 and served two-term as his VP. Since 1993, Atiku has continued to eye the presidential seat and is set to battle it with other contenders on February 25, 2023. In 1993, he contested the Social Democratic Party (SDP) presidential primaries and lost to Moshood Abiola. Also, in 2007, he was a presidential candidate for the Action Congress. In 2011, he also contested the presidential primaries of the PDP and lost to Goodluck Jonathan who was the incumbent. In 2014, he joined the APC and lost to Muhammadu Buhari at the party’

Verdict:
FALSE

Date:
27-01-2023

URL:
https://factcheckhub.com/atikus-wife-titi-abubakar-makes-false-claim-about-the-office-of-nigerias-first-lady/

('No Yoruba woman has bore the title of First Lady in Nigerian Presidency.\n', 'Findings by The FactCheckHub show that the claim is FALSE. Since 1963, during the administration of the country’s first president, Nnamdi Azikwe till date, Nigeria has had 15 First Ladies in the Presidency. They include: Flora Azikiwe (1963-1966), Victoria Aguiyi-Ironsi (1966-1966), Victoria Gowon (1966-1975), Ajoke Muhammed (1975-1976), Esther Oluremi Obasanjo (1976-1979, Hadiza Shehu Shagari (1979-1983), Safinatu Buhari (1983-1985), and Maryam Babangida (1985-1993). Others are Margaret Shonekan (1993), Maryam Abacha (1993-1998), Fati Lami Abubakar (1998-1999), Stella Obasanjo (1999-2005), Turai Yar’Adua (2007-2010), Patience Jonathan (2010-2015), and Aisha Muhammadu Buhari (2015 till date). In the last 59 years, Nigeria has had at least three First Ladies fr

Unwanted page

Unwanted page

Claim:
Visuals show image of Peter Obi on bags of rice branded with PDP logo.


Evidence:
Checks by The FactCheckHub show that the claim is FALSE. The purported video shows a scene where some bags of rice branded with PDP logo and the names of some contestants under the party were being loaded into a truck. When our researcher did a keyframe analysis of the footage using InVid – a video verification tool, findings show the branded bags of rice had on them names of some PDP candidates who are contesting for various elective positions in Delta state as well as that of Atiku Abubakar and Ifeanyi Okowa, the party’s presidential candidate and his vice, respectively. A clearer pictures of branded bags of rice/ Source: Facebook Names seen on the bags include: Hilary Obioma Igbebulem, secretary to the Delta state governor; Atiku Abubakar, presidential candidate; Ifeanyi Okowa, vice-presidential candidate; Sheriff Oborevwori, PDP governorship candidate in Delta sta

Claim:
Reno Omokri joins APC supporters in London to campaign for Bola Tinubu at Chatham House.

Evidence:
Findings by The FactCheckHub show the claim is FALSE. The Chatham House is an independent policy institute headquartered in London. The group, also recognised as the Royal Institute of International Affairs, has become a place most presidential political candidates visit to showcase their agenda, especially as they get close to the election period. A former Nigerian Vice President, Atiku Abubakar, was at the Chatham House in April 2018. Meanwhile, an analysis of the video showed that contrary to the claim, Reno was actually using derogatory words against Tinubu in a protest at the same location (Chatham House). Screenshot of the tweeted video showing Reno Omokri was protesting against Tinubu’s candidacy in London on Monday with a placard that reads: ‘No to a drug baron in Aso Rock.’ Research showed that there were two camps around the Chatham House, a camp loyal to Tinubu, and Ren

In [19]:
len(field_list1)

35

## fc_links2

In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

def replace_blanks_and_linebreaks(s):
        s = s.replace('\xa0', ' ')
        s = s.replace('\n', ' ')
        s = s.replace('\t', ' ')
        s = s.replace('\xa0\n', ' ')
        s = re.sub(r'\s+', ' ', s)
        return s

claims, verdicts, justifications, dates, urls = [],[],[],[],[]

field_list2 = []
possible_labels = ['true', 'mostly true', 'false', 'mostly false', 'miscaptioned', 'misleading', 'incorrect']

#url = 'https://factcheckhub.com/anambradecides2021image-does-not-show-apc-chairman-collapsing-after-losing-lg-to-pdp/'
api_key = '27308361-0f5c-4501-98d2-b9b108fdef85'

for url in fc_links2:
    try:
        url_tuple = ()
        response = requests.get(url='https://proxy.scrapeops.io/v1/',
                            params={'api_key': api_key,
                                    'url': url,
                                    'render_js': True})
        src = response.content     
        soup = BeautifulSoup(src, 'html.parser')

        # # # # # # # # # # # # # # # # #
        # get all text from the article #
        # # # # # # # # # # # # # # # # #

        article_text = replace_blanks_and_linebreaks(soup.text).strip()



        # # # # # # # # #
        # extract claim #
        # # # # # # # # #

        pattern_claim = re.compile(r'.*CLAIM.*')
        claim_header = soup.find(text=pattern_claim)
        #claim_header = soup.find(text='THE CLAIM:')
        claim = claim_header.find_next('p').text
        #claims.append(claim)
        if len(claim) > 1:
            url_tuple = url_tuple + (claim,)
            print(f'Claim:\n{claim}\n')



        # # # # # # # # # # #
        # extract evidence  #
        # # # # # # # # # # #

        start_word = "FINDINGS"
        end_word = "VERDICT"
        pattern = re.compile(f"{start_word}(.*?){end_word}", re.DOTALL)
        evidence = pattern.findall(article_text)[0].split('THE ')[0].strip()
        if evidence.startswith(': '): evidence = evidence.split(': ')[1]
        #justifications.append(evidence)
        if len(evidence) > 1:
            url_tuple = url_tuple + (evidence,)
            print(f'Evidence:\n{evidence}\n')


        # # # # # # # # # #
        # extract verdict #
        # # # # # # # # # #

        pattern_verdict = re.compile(r'.*VERDICT.*')
        verdict_header = soup.find(text=pattern_verdict)
        #claim_header = soup.find(text='THE CLAIM:')
        verdict_par = verdict_header.find_next('p').text
        pattern_verdict_is = re.compile(r'\bis\b\s+(\w+)')
        match = pattern_verdict_is.search(verdict_par)
        verdict = match.group(1)
        #verdicts.append(verdict)
        if verdict.lower() in possible_labels and len(verdict) > 1:
            url_tuple = url_tuple + (verdict,)
            print(f'Verdict:\n{verdict}\n')



        # # # # # # # # #
        # extract date  #
        # # # # # # # # #

        date = str(soup.find('header', class_='entry-header').text.split('By')[-1].split('on')[-1]).strip()
        dt_object = datetime.strptime(date, '%B %d, %Y')
        formatted_date = dt_object.strftime('%d-%m-%Y')
        #dates.append(formatted_date)
        if len(formatted_date) > 1:
            url_tuple = url_tuple + (formatted_date,)
            print(f'Date:\n{formatted_date}\n')



        # # # # # # # # #
        #  extract url  #
        # # # # # # # # #
        
        #urls.append(url)
        if len(url) > 1:
            url_tuple = url_tuple + (url,)
            print(f'URL:\n{url}\n')

        
        if len(url_tuple) == 5:
            print(url_tuple)
            field_list2.append(url_tuple)
        
    except:
        print('Unwanted page\n')
        continue

Claim:
Adebayo claimed that 80 per cent of Nigeria’s crude oil is being stolen.

Evidence:
Findings by the Nigerian Fact-checkers Coalition show that the claim is FALSE. The coalition did a live check during the interaction and found that the amount of crude oil the country lost to theft and vandalism was way lesser than 80 per cent. The official figures cited from the Nigerian Upstream Petroleum Regulatory Commission (NUPRC) revealed that Nigeria lost 115,000 barrels of crude oil per day (bpd) between January and February 2022. The data shows that the crude oil loss was less than 10 per cent of the country’s regular daily production of about 1.5 million barrels a day.

Date:
19-11-2022

URL:
https://factcheckhub.com/fact-checking-adewole-adebayos-claims-at-the-candidates-presidential-town-hall-series/

Unwanted page

Claim:
Wike defects to APC.

Evidence:
Findings by The FactCheckHub show that the claim is FALSE. Checks using keyword searches online show that the video had surfaced on

Claim:
That photo shows Bola Tinubu watching Peter Obi on television.

Evidence:
Findings by The FactCheckHub show that the claim is FALSE. Using Google Reverse Image Search, checks revealed that the photo has been manipulated. The original photo shows Bola Tinubu staring at a TV screen. The original image shows Tinubu in an aeroplane staring at a TV screen on his way from the United Kingdom to Nigeria. However, there was nothing showing on the screen. This is not the first time that Charly Boy has shared misleading claims online. The FactCheckHub had earlier reported how he shared a misleading video with a claim that it shows Peter Obi’s supporters in boats holding a campaign rally in riverine areas. When The FactCheckHub subjected the video to verification, it revealed that the claim was FALSE. The FactCheckHub also reported how he made a false claim about the electronic transmission of election results by the Independent National Electoral Commission (INEC).

Verdict:
MISLEADING

Da

Unwanted page

Claim:
Checks by The FactCheckHub show that Charly Boy’s claim is FALSE.

Evidence:
Checks by The FactCheckHub show that Charly Boy’s claim is FALSE. Charly Boy indeed referred to a recent interview granted by the INEC’s National Commissioner and Chairman, Information and Voter Education Committee, Festus Okoye, where he said election results would be collated manually. Okoye said the election would be collated manually in the interviews but explained the difference between collation and transmission of results. He states, “There is a marked difference between the transfer/transmission of results and the collation. Section 50(2) of the Electoral Act, 2022 gives the commission the absolute discretion to determine the mode and procedure of voting in an election and the transmission of election results. “Sections 60 and 62 of the Electoral Act govern post-election procedure and collation of election results. Section 60(1) of the Act provides that the presiding officer shall

Claim:




Evidence:
Checks by The FactCheckHub show that the claim is FALSE. Wole Oladimeji, the media aide to the Labour Party gubernatorial candidate in Osun State, told The FactCheckHub that Yusuf Lasun is still so much in the race. “There is nothing like that, it is a figment of their imagination,” Oladimeji says in a telephone conversation. Similarly, Lasun was seen casting his vote on Saturday in Osun state where he also warned against vote-buying, Channels TV reported. His political party, of which he was the flagbearer, was also on the ballot papers used during the election. “I have not stepped down, and I will never step down for anyone,” Lasun said.

Verdict:
FALSE

Date:
16-07-2022

URL:
https://factcheckhub.com/osun-decides-2022-claim-that-yusuf-lasun-has-withdrawn-from-governorship-poll-is-false/

('\n\n', 'Checks by The FactCheckHub show that the claim is FALSE. Wole Oladimeji, the media aide to the Labour Party gubernatorial candidate in Osun State, told The FactCheckHu

Claim:
That Osun state has not had any serious case of banditry, kidnapping and armed robbery since Gboyega Oyetola became governor of the state.

Evidence:
Checks by The FactCheckHub show that the claim is MOSTLY FALSE. While it is true that Osun is one of the states in Nigeria with few cases of violent attacks, findings by The FactCheckHub show that several cases of armed attacks have been recorded in the state under the Oyetola-led government. Preceded by Rauf Aregbesola, Oyetola assumed office as the governor of Osun State on the 27th of November, 2018. Analysing data from Nigeria Security Tracker (NST), a project of the Council on Foreign Relations’ Africa programme documents and maps violence in Nigeria that is motivated by political, economic, or social grievances, The FactCheckHub filtered out cases of armed violence recorded in Osun state between November 2018 when Oyetola assumed office up till July 2022 when he made the statement. The data shows that more than 100 cases of a

...

Claim:
Dutch National born in Nigeria Wiebe Boer declares his intention to contest for Plateau governorship.

Evidence:
Findings by The FactCheckHub show that the image of the man in the poster is that of Wiebe Boer, Chief Executive Officer of All On, an off-grid energy investment company in Nigeria. Boer was born and brought up in Jos, North Central Nigeria to Dutch missionary parents, reports say. His father and mother came to Taraba State, Nigeria, in 1966 as missionaries to spread the gospel and engage with the Nigerian church and leadership. He also authored a book: “A Story of Heroes and Epics: The History of Football in Nigeria.” However, checks by The FactCheckHub show that the claim indicating that he has joined the Plateau governorship race is FALSE. When contacted, most social media handles spreading ‘the news’ could not provide any source to where he made the declaration. A keyword search on the internet also gave no result of where he made the declaration. In a statement p

Claim:
First time in Nigeria a group of people crowd-funded to buy the expression of interest and nomination forms for a politician.

Evidence:
Findings by the FactCheckHub show that before Saraki, there have been many groups who reportedly raised funds to purchase nomination and expression of interest forms for Nigerian politicians to contest elective positions. Here are a few instances of politicians who received the same gesture to contest for a presidential seat: (1) Atiku Abubakar: On the 18th of March, 2022 – a few days before Saraki’s announcement, a business forum named North East Business Forum purchased the 2023 Presidential Nomination and Expression of Interest forms for the former Vice President, Atiku Abubakar. The group, who later presented the forms to Atiku, a presidential hopeful at his Abuja residence, said the gesture was in fulfilment of their promise in 2021 to purchase the forms for him. The former president reportedly received the same gesture in 2018. (2) Muhamm

Claim:
No voting in Aguleri, as everyone is indoor.

Unwanted page

Claim:
Andy Uba sponsored the Prison Reform Bill

Unwanted page

Claim:
Anambra State has the lowest unemployment rate  13 per cent and the lowest underemployment rate at 17 per cent of any state in Nigeria.

Evidence:
Findings by The FactCheckHub show that the claims are misleading because Soludo made reference to an old set of data. The data he referenced are actually from the second quarter (Q2) of 2020. The 2020 Q2 data showed that Anambra has the lowest of unemployment rate at 13.1 per cent and also the lowest underemployment rate at 17 per cent. Screenshot of National Bureau of Statistics document, unemployment and underemployment Q2, 2020. After the Q2 labour statistics, the NBS released the Q4 statistics. In the Q4 data, Anambra’s unemployment rate is 44.23 per cent while the underemployment rate is 16.48 per cent. Furthermore, Anambra was not the lowest as Osun has 11.65 per cent unemployment rate, while Lagos

Claim:
That the ZLP candidate in the Ondo state gubernatorial election, Agboola Ajayi, withdrew from the poll.

Evidence:
The FactCheckHub reached out to Babatope Okeowo, the Chief Press Secretary to the Deputy Governor of Ondo State, Agboola Ajayi, to verify the image and viral WhatsApp post. In his brief response to the FactCheckHub’s enquiry via his WhatsApp number, he stated that it’s “fake news”. Similarly, Comrade Allen Sowore, the Media Adviser to the Deputy Governor (Agboola Ajayi), in a press statement on Saturday also debunked the post and withdrawal letter. “Again, the attention of the Deputy Governor and candidate of the Zenith Labour Party (ZLP), Agboola Ajayi has been drawn to a forged resignation letter from the Governorship Election holding today, Saturday- 10 October, 2020…” “For the umpteenth time, and for the purposes of clarity and emphasis, the Deputy Governor and candidate of ZLP, Agboola Ajayi will not resign or back out from the race he’s winning ultimately. “Th

Claim:




Evidence:
The FactCheckHub checked the final list released by Nigeria’s Independent National Electoral Commission (INEC) to verify if the party and its candidate have been removed from the contest as claimed in the viral message but the name of the APC candidate was present. Osagie Andrew Ize-Iyamu (58) is number 7 on the list. The name of his deputy, Abudu Ganiyu Audu (50) was also in the INEC list. He was listed number 8 in the document signed by Rose Oriaran-Anthony, Secretary to the electoral commission. Unlike what is obtainable in the Nigerian media industry, the press statement was not signed by anyone. Moreover, no such information was shared by official channels of the APC and the INEC. Reports also affirmed Ize-Iyamu voted at Ugboko Ward 4, Unit 26, Orhionmwon Local Government Area (LGA) of the state at about 9:25am on September 19, 2020. In addition, Ize-Iyamu told the press that he is confident of winning after casting his vote on Saturday. Earlier, the FactCheck

In [7]:
len(field_list2)
field_list2

[('Wike defects to APC.',
  'Findings by The FactCheckHub show that the claim is FALSE. Checks using keyword searches online show that the video had surfaced online with similar claims around June 2022 after the PDP presidential primary election. Analysis of the video shows that it was cut from Wike’s interview with Arise TV broadcast on July 2021. In the original video, Wike was reacting to the defection of some governors and legislators from the PDP to the ruling APC. Prior to the interview, PDP had lost three governors and other party members to the APC in the space of seven months. The River State governor responded to the question by the Arise TV anchor, Oseni Rufai, who asked if the reason members of the party, especially from the South-East, were leaving the party was because the party did not consider zoning. In the full video (29:00-32:00 minutes), Wike argued that the defectors could not justify their reasons for leaving the party. He added that their reasons should be in the

### MANUALLY ADDED CLAIMS

In [29]:
manual = [('A Twitter user, @omoelerinjare, posted a video stating that the INEC collation officer deployed to Obingwa LGA in Abia State was brutalised by PDP thugs for not allowing them to change election results.', 'According to Radio Nigeria’s report, INEC’s office in Obingwa Local Government of Abia state was attacked last Sunday. Some of its officials were held hostage in connection with the collation of results. The incident has prompted INEC’s review of the result. However, the NFC could not ascertain if they were thugs from any particular political party.', 'PARTLY CORRECT', '21-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'),
          ('A broadcast message circulated on social media, particularly WhatsApp, implies that the INEC’s presiding Officer for Abia state, Professor Nnenna Oti, stated that the INEC Chairman, Professor Mahmood Yakubu “directed” her to influence election results in Abia state.', 'The alleged statement is found to have been doctored and altered. Although Ms Oti made a statement before calling on her administrative secretary to read the INEC’s press release suspending the election, she never insinuated foul play, as stated in the viral post. A 13-minute-long video report by TVC contains her actual statement. The INEC chairman and her administrative secretary debunked the claims when the NFC contacted them. Rotimi Oyekanmi, Chief Press Secretary to INEC, has also issued a rebuttal debunking the claim.', 'INCORRECT', '21-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'),
          ('Twitter users have shared a video claiming that police officers colluded with PDP thugs and INEC ad-hoc officials to whisk away ballot boxes in ward 20, polling unit 042, Port Harcourt Local Government Area (PHALGA) of Rivers state, during the governorship elections.', 'From the NFC’s footage review, the police officers did not engage the INEC officers. Also, CODE observer Evelyn Williams, in Rivers state, confirmed to the NFC that thugs snatched ballot boxes in some local government wards in Obio-Akpor and PHALGA. Still, there were no reports of police officers’ involvement. Meanwhile, the NFC discovered that voting results from the polling unit had been uploaded on the IReV portal. According to the results, PDP polled 85 votes, LP got six, and Action Democratic Party (ADP) scored four. The Social Democratic Party (SDP) and the Action Party (A) received two votes each.', 'UNPROVEN', '21-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Naija news report claiming that Anambra state governor, Chukwuma Soludo, has threatened state residents that “there will be consequences for not voting APGA” has been circulating on social media.', 'The report is based on a viral video, which shows Soludo discussing with a few people in Igbo language, standing in front of the Anambra State government House. The governor told a story claiming that Micheal Okpara, the premier of the Eastern Region in the First Republic, unearthed pipes his government laid in Inyi because the town did not vote for his party NCNC. Soludo concluded, “there must be a consequence, and there must be a reward.” We analysed the video on InVID and its metadata confirmed that it is recent. In a different video, Soludo promised to cooperate with only APGA state assembly members.', 'CORRECT', '21-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A social media user, @JohnFanimokun, shared a video on Twitter claiming that the All Progressives Grand Alliance and Labour Party supporters clashed in Anambra state on Saturday.', 'Using Google Earth, the NFC performed a geo-location on the video and discovered it was captured at Shadia Onanuga street, Abule-Ado, in Lagos state. Also, an INVID analysis of the video showed that it appeared recently, during the elections. Also, the female voice-over alleged that the violence distracted voters.', 'INCORRECT', '20-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'),
          ('A Twitter user, Habibu Abdulwahab (@sarkiabdulwahab), claims that at just 23 years of age, Mr Muhammad Adamu Oyanki has been elected to represent Doma North at the Nasarawa state House of Assembly.', 'Based on the final list of House of Assembly contestants released by INEC and published by Premium Times, Mr Muhammad Adamu Oyanki, contesting under PDP, is 26 years old, not 23. As of 10 am today, INEC is yet to release the results of the Nasarawa state HOA elections.', 'INCORRECT', '20-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user, @chukzwrld, shared a video claiming that Enugu residents camped in the INEC office on Monday, 20 March 2023, “demanding the restoration of their stolen mandate!”', 'In the video, protesters sang and prayed in Igbo, a language widely spoken in Enugu state, and they held posters of the Labour Party (LP) governorship candidate in Enugu, Chijioke Edeoga. Enugu-based journalists confirmed the incident saying they are LP supporters protesting due to allegations of manipulation of the governorship election results of two outstanding local government areas – Nsukka and Nkanu East. In response, the Independent National Electoral Commission (INEC) announced the suspension of collation in those local government areas.', 'CORRECT', '20-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'),
          ('The Nigerian Tribune reports that Peter Mbah of the Peoples Democratic Party (PDP) has won the Enugu State gubernatorial election.', 'The Independent National Electoral Commission (INEC) has suspended the collation of results in the state. The commission said the suspension was to review results from two outstanding LGAs of Nsukka and Nkanu East. As of 3:29pm on Monday, INEC had yet to complete the review. It has also not announced the winner as claimed.', 'MISLEADING', '20-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'),
          ('Twitter users claim Alex Otti, Abia state governorship candidate of the Labour Party, has been declared the winner.', 'Voting data available on the result viewing portal (IReV) of the Independent National Electoral Commission (INEC) shows results from 1,844 polling units in the state, out of 4,062, have been uploaded as of 9:30pm on Saturday. Also, the purported declaration of the candidate as the election winner has not been published by any credible media platform. The CODE election observer in the state, Ugochukwu Favour, told the NFC on Saturday at 11pm that collation is still ongoing at the ward level. Thus, a winner is yet to be declared.', 'INCORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user, IperuBoy (@Iperu_Boy), claims that the APC gubernatorial candidate in Ogun State, Dapo Abiodun, led thugs to snatch ballot boxes at Iperu, Ikenne local government.', 'Punch Journalist Ojuroungbe Sodiq, who was at the scene, stated that although thugs snatched ballot boxes, it was difficult to ascertain who was responsible. Also, Mr Abiodun did not appear in the pictures attached to the tweet.', 'MISLEADING', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user claims Hon Anthony Chinasa Abiola is Yoruba representing Umuahia Central Constituency in Abia State House of Assembly under the Labour Party.', 'In a 1:10 minute YouTube clip by Television Continental (TVC News), Mr Chinasa clarified that he is not Yoruba. Chinasa, who spoke from 0:34 onwards, added that he is not of mixed ethnicity but fully Igbo.', 'INCORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('Twitter user, Kelechi Favour (@kclove_diamond), shared an AIT video capturing a staff of the Independent National Electoral Commission (INEC) caught with ballot papers and election materials in Olamaboro, Kogi State.', 'The NFC contacted a Connected Development (CODE) observer, Umar Muhammed, in Kogi, who confirmed that the incident occurred. He explained that the man alleged a local government Chairman directed him to commit the act. The video is featured on AIT Live’s YouTube page.', 'CORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user, Chinaza #PeterObi-HypeMan (@MuchTalksBlog1), claims that Wasiu Ayinde, Nigerian Fuji singer, was among the thugs who came to disrupt the election on Saturday.', 'The one-minute-long video the claimant shared did not show that the Fuji singer was present at the location. Additionally, no reports from any credible media platform suggest or disclose Mr Ayinde’s role in ballot snatching during yesterday’s election. Reacting to the allegation, Kunle Rasheed, the singer’s spokesperson, debunked the claims on his Facebook page, reiterating that Mr Ayinde will never partake in such an act nor endorse it.', 'INCORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user, TIFE (@ZeekiHodl), claims that Olumide Oworu is not the INEC approved Labour Party’s candidate for Surulere, Lagos state House of Assembly.', 'Premium Times’ report revealed that Mr Oworu was not included in INEC’s final list of contestants. This was also confirmed by Daily Trust’s report, which stated that Mr Adebayo Bode is the approved LP’s candidate for Surulere 1 Constituency.', 'CORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('Viral social media posts claim a young man defeated the incumbent speaker, Yobe State House of Assembly, Ahmed Lawan Mirwa, in the polls.', 'A report by TheCable confirms that the current speaker of the Yobe State House of Assembly, Mr Mirwa, lost his reelection bid for the Nguru state constituency II. Musa Lawan,  the candidate of the Peoples Democratic Party (PDP), won instead. Mr Lawan polled 6,648 votes, while Mr Mirwa got 6,466 votes. Mr Lawan’s details on the INEC list showed that he is 32 years old.', 'CORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user, @Theoladeledada, claims that Oladipupo Olatunde is the governor-elect of Ogun state.', 'Checks on the Independent National Electoral Commission (INEC) website show that Oladipupo Olatunde Adebutu is the governorship candidate of the PDP in Ogun state. When the NFC contacted Oluwaseyi Afolabi, CODE observer in Ogun state, she clarified that as of 3:30 pm today, no winner had been declared at the INEC collation centre in the state. Meanwhile, voting data on INEC’s result viewing portal (IReV) shows that, as of 3.05 pm, results from 4,931 polling units out of 5,042 in the state have been uploaded.', 'INCORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user @goldmyneTV posted a video of people celebrating in Yola, claiming that Aishatu Dahiru, popularly known as Binani, has become the first female governor to be elected.', 'INEC did not announce Binani as the governor-elect of Adamawa state. According to the Premium Times report as of 4pm on Sunday, only ten local governments have been announced out of Adamawa’s 21 local governments. In the report, PDP’s candidate and incumbent governor, Ahmadu Fintiri, currently leads with 186,536 votes, while APC’s Binani has 179,264.', 'INCORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('Social media users claim a man named Chude has been kidnapped by the State Criminal Investigation Department in Anambra state.', 'Chude (Nnamdi Chude), is a vocal voice for the Labour Party and Peter Obi on Twitter. The Public Relations Officer of Anambra State Police Command,  Ikenganyia Tochukwu has not replied the NFC’s messages. However, Prof Chukwuma Charles Soludo, the governor of Anambra State, confirmed that Chude was arrested on 18 March 2023 in Anambra by the Cyber Crime Unit of Interpol and taken to Abuja.', 'CORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user, @DeeOneAyekooto, claims that Jigawa State’s final results have been declared as follows: APC 460,302; PDP 275,776.', 'As of 6:29pm on Sunday, results on INEC’s IReV show that only 11 LGA results have been uploaded out of 27 LGAs. Also, no credible media platforms have reported the final results in Jigawa state.', 'INCORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user, @ObrusMados, claims that Shittu Rukayat is the youngest member of the state House of Assembly in the whole of Nigeria.','Yusuf Adebisi, a lawmaker in Oyo State House of Assembly, was the youngest state legislator when he was elected in 2019 at age 25 to represent Ibadan South West Constituency 1. However, Shittu Rukayat, 26, who was declared winner of the Kwara State House of Assembly election, is the youngest state legislator to be elected in 2023.', 'CORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('Twitter users have shared a photo of people holding placards and protesting against Igbos in Lagos. The photo was shared in connection with pockets of clashes in Lagos during the governorship election on Saturday.', 'TinEye and Google reverse image searches show the picture has been on the internet since 2015 and is unrelated to Saturday’s elections. The photo was shared on Nairaland, a Nigerian internet forum, in 2015.', ' MISLEADING', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user, @Mmadudili, claims that Labour Party’s candidate, Kelvin Chukwu, has been declared winner of Enugu East Senatorial election.', 'The Punch Newspaper has reported that former Enugu State governor and Senator, Chimaroke Nnamani, lost his re-election bid to Kelvin Chukwu of the Labour Party.  Leadership Newspaper also confirms Labour Party’s victory.', 'CORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'), 
          ('A Twitter user, Love Chijioke (@Realsnowlove1), suggests the PDP scored 30,000 votes in the Enugu State gubernatorial election when less than 13,323 votes were cast in the presidential elections.', 'The latest report by PREMIUM TIMES (published around 4pm on Sunday) on the Enugu gubernatorial elections indicates that results are still being collated. Results gathered from eight (8) out of the 17 local governments show that PDP had at the time of the report polled 66,907 votes. Meanwhile, PREMIUM TIMES reported  that a total number of 468,891 votes were cast in the presidential election in Enugu state. The PUNCH newspapers also recorded the same report, stating 456,424 as valid votes, 12, 467 invalid, and 468, 891 as the total votes cast.', 'INCORRECT', '19-03-2023', 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/')]

### JOIN ALL TUPLES

In [20]:
field_list = field_list1 + field_list2
len(field_list)

71

In [30]:
field_list = field_list + manual

### CREATE DATAFRAME

In [32]:
import pandas as pd
df = pd.DataFrame(field_list, columns=['Claim', 'Evidence', 'Label', 'Date', 'URL'])
df['Evidence'], df['Label'] = df['Label'], df['Evidence']
df = df.rename(columns={'Claim':'Claim', 'Evidence':'Label', 'Label':'Evidence', 'Date':'Date', 'URL':'URL'})[['Claim', 'Evidence', 'Label', 'Date', 'URL']]
pd.options.display.max_rows = None
df

,Claim,Evidence,Label,Date,URL
0,Electoral law says contestants for House of Re...,Checks by the Nigerian Fact-checkers Coalition...,INCORRECT,17-02-2023,https://factcheckhub.com/does-nigerias-elector...
1,"Gbadebo Rhodes-Vivour, the governorship candid...","According to several media reports, President ...",INCORRECT,05-02-2023,https://factcheckhub.com/fact-checking-three-l...
2,Peter Obi claimed that over 130 million live i...,According to a recent report by the National B...,TRUE,25-11-2022,https://factcheckhub.com/fact-checking-obis-cl...
3,Kwankwaso claimed that he never borrowed money...,"According to a report by Premium Times, Kano s...",FALSE,22-11-2022,https://factcheckhub.com/fact-checking-kwankwa...
4,Obasanjo’s administration spent $16 billion do...,There were allegations that President Olusegun...,FALSE,19-11-2022,https://factcheckhub.com/fact-checking-sowores...
5,Video shows INEC REC brutalized in Adamawa state.,Findings by The FactCheckHub show that the cla...,FALSE,16-04-2023,https://factcheckhub.com/video-does-not-show-i...
6,"Money, textile fabrics and foodstuffs were dis...",Findings by the FactCheckHub show that this cl...,TRUE,16-04-2023,https://factcheckhub.com/claim-that-money-fabr...
7,Atiku Abubakar has retired from politics and c...,Findings by The FactCheckHub revealed that the...,FALSE,09-04-2023,https://factcheckhub.com/false-claim-circulate...
8,US criticizes Nigeria’s 2023 presidential elec...,Findings by The FactCheckHub show that the cla...,MISLEADING,09-04-2023,https://factcheckhub.com/no-us-government-did-...
9,Court of Appeal declares Gboyega Oyetola as th...,Findings by The FactCheckHub show that the cla...,FALSE,25-03-2023,https://factcheckhub.com/osun-election-false-c...


In [33]:
df.to_csv("factcheckhub.csv", index=False)

<a id='content'></a>

## Content scraping

In [14]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def replace_blanks_and_linebreaks(s):
    s = s.replace('\xa0', ' ')
    s = s.replace('\n', ' ')
    s = s.replace('\xa0\n', ' ')
    return s

claims, verdicts, justifications, dates, urls = [],[],[],[],[]
for url in fc_links:
    try:        
        response = requests.get(url='https://proxy.scrapeops.io/v1/',
                            params={'api_key': '3ea90454-6ad6-4316-994c-642edb2c4ebd',
                                    'url': url,
                                    'render_js': True})
        src = response.content     
        soup = BeautifulSoup(src, 'html.parser')
        whole_page = soup.find_all('div', class_='entry-content')
        whole_page = BeautifulSoup(str(whole_page), 'html.parser')
        paragraphs = whole_page.find_all('p')
        for i in paragraphs:
            if i.text.startswith('CLAIM: '):
                if 'href' in str(i):
                    i_raw = str(i)
                    soup2 = BeautifulSoup(i_raw, 'html.parser')
                    claim_links = [a_tag.get('href') for a_tag in soup2.find_all('a')]
                    urls.append(claim_links[0])
                else:
                    urls.append(url)
                claims.append(replace_blanks_and_linebreaks(i.text.split('CLAIM: ')[1]))
            elif i.text.startswith('FINDINGS: '):
                justifications.append(replace_blanks_and_linebreaks(i.text.split('FINDINGS: ')[1]))
            elif i.text.startswith('FINDING: '):
                justifications.append(replace_blanks_and_linebreaks(i.text.split('FINDING: ')[1]))
            elif i.text.startswith('VERDICT: '):
                verdicts.append(replace_blanks_and_linebreaks(i.text.split('VERDICT: ')[1]))
            elif i.text.startswith('DATE PUBLISHED: '):
                dates.append(replace_blanks_and_linebreaks(i.text.split('DATE PUBLISHED: ')[1]))
    except:
        print('Unwanted page')
        continue

### La celda con el algoritmo principal tarda unos 7m 55s en ejecutar

In [70]:
len(claims) == len(verdicts) == len(justifications) == len(dates) == len(urls)

False

In [79]:
for index,i in enumerate(justifications):
    print(f'{index}\t {i} \n')

0	 As of 5pm on Tuesday (March 21), INEC is yet to declare the winner of the gubernatorial election in Enugu state. A press statement released by the commission says collation has been suspended in the state. Vanguard, Thisday live, and Tribune online all reported the suspension of the collation. 

1	 According to Radio Nigeria’s report, INEC’s office in Obingwa Local Government of Abia state was attacked last Sunday. Some of its officials were held hostage in connection with the collation of results. The incident has prompted INEC’s review of the result. 

2	 The alleged statement is found to have been doctored and altered. Although Ms Oti made a statement before calling on her administrative secretary to read the INEC’s press release suspending the election, she never insinuated foul play, as stated in the viral post. A 13-minute-long video report by TVC contains her actual statement. The INEC chairman and her administrative secretary debunked the claims when the NFC contacted them. 

In [151]:
url = 'https://factcheckhub.com/live-check-collation-of-votes-for-nigerias-2023-governorship-and-state-assembly-elections-ongoing/'
try:        
    response = requests.get(url='https://proxy.scrapeops.io/v1/',
                        params={'api_key': '22a808fd-7dd1-412a-9f92-5ae5a25d0778',
                                'url': url,
                                'render_js': True})
    src = response.content     
    soup = BeautifulSoup(src, 'html.parser')
    whole_page = soup.find_all('div', class_='entry-content')
    whole_page = BeautifulSoup(str(whole_page), 'html.parser')
    paragraphs = whole_page.find_all('p')
    for i in paragraphs:
        
        if i.text.startswith('CLAIM: '):
            if 'href' in str(i):
                i_raw = str(i)
                soup2 = BeautifulSoup(i_raw, 'html.parser')
                claim_links = [a_tag.get('href') for a_tag in soup2.find_all('a')]
                urls.append(claim_links[0])
            else:
                urls.append(url)
            claims.append(replace_blanks_and_linebreaks(i.text.split('CLAIM: ')[1]))
        
        elif i.text.startswith('FINDINGS: '):
            justifying_text = replace_blanks_and_linebreaks(i.text.split('FINDINGS: ')[1])
            justifications.append(justifying_text)
        
        elif i.text.startswith('FINDING: '):
            justifying_text = replace_blanks_and_linebreaks(i.text.split('FINDING: ')[1])
            justifications.append(justifying_text)
        
        elif i.text.startswith('VERDICT: '):
            verdicts.append(replace_blanks_and_linebreaks(i.text.split('VERDICT: ')[1]))
        
        elif i.text.startswith('DATE PUBLISHED: '):
            dates.append(replace_blanks_and_linebreaks(i.text.split('DATE PUBLISHED: ')[1]))
except:
    print('Unwanted page')

In [203]:
for index,i in enumerate(paragraphs):
    if 'FINDINGS' in i.text:
        print(index)

10
16
23
29
36
43
50
56
62
68
75
80
86
92
99
112
120
126
133
139
146
152
158


In [219]:
for index,i in enumerate(paragraphs):
    print(i.text)

Collation of election results is ongoing following the conduct of Nigeria’s 2023 Governorship and State Assembly Elections.
Nigerians on Saturday, March 18, 2023 voted to elect their governors in 28 states as well as state lawmakers into 36 State Houses of Assembly across the country.

All the 18 registered political parties in Nigeria fielded over 400 governorship candidates in the 28 states during the poll.
The Nigerian Fact-checkers’ Coalition (NFC) brings you live-check of election-related misinformation and disinformation spreading online, as vote collation begins across the nation.
Our team of researchers, fact-checkers, social media monitors, editors, misinformation & disinformation experts drawn from the coalition partners, are working from the coalition’s Election Situation Rooms located in Abuja, Lagos and UK, to debunk misinformation and disinformation contents targeted at inciting violence or influencing the outcome of elections.
Do you have any claim you want us to fact-ch

In [227]:
paragraphs_aux = paragraphs.split('\n\n')  # Split text into paragraphs
for i,para in enumerate(paragraphs_aux):
    if 'FINDINGS' in para.text:
        print(f'{i},{para.text}')
  #      result.append(para.text)
   #     j = i + 1
    #    while 'VERDICT' not in paragraphs[j].text:
     #       result.append(paragraphs[j].text)
       #     j += 1
      #  break

AttributeError: 'str' object has no attribute 'text'

In [215]:
result

['FINDINGS: As of 5pm on Tuesday (March 21), INEC is yet to declare the winner of the gubernatorial election in Enugu state. A press statement released by the commission says collation has been suspended in the state. Vanguard, Thisday live, and Tribune online all reported the suspension of the collation.']

In [259]:
result_par = []

for i,para in enumerate(paragraphs):
    if para.text.startswith('FINDINGS: '):
        #print(f'{i},{para.text}\n')
        text = replace_blanks_and_linebreaks(para.text.split('FINDINGS: ')[1])
        result_par.append(text)
        j = i + 1
        #print(i)
       # print(paragraphs[i])
       # print(j)
       # print(paragraphs[j])
        #while 'VERDICT' not in paragraphs[j]:
           # result_par.append(paragraphs[j])
         #   j += 1
      #  break
   # print(' '.join(result_par))  # Print the result as a single string separated by two newlines
            
            
            
for i in result_par: print(f'{i}\n')

As of 5pm on Tuesday (March 21), INEC is yet to declare the winner of the gubernatorial election in Enugu state. A press statement released by the commission says collation has been suspended in the state. Vanguard, Thisday live, and Tribune online all reported the suspension of the collation.

According to Radio Nigeria’s report, INEC’s office in Obingwa Local Government of Abia state was attacked last Sunday. Some of its officials were held hostage in connection with the collation of results. The incident has prompted INEC’s review of the result.

The alleged statement is found to have been doctored and altered. Although Ms Oti made a statement before calling on her administrative secretary to read the INEC’s press release suspending the election, she never insinuated foul play, as stated in the viral post. A 13-minute-long video report by TVC contains her actual statement. The INEC chairman and her administrative secretary debunked the claims when the NFC contacted them. Rotimi Oyek

<a id='export'></a>

## Saving to CSV file

In [133]:
everything = list(zip(claims, veredicts, justifications, dates, urls))

In [136]:
import pandas as pd
df = pd.DataFrame.from_records(everything, columns=["Claim", "Label", "Evidence", "Date", "URL"])
pd.options.display.max_rows = None
df

,Claim,Label,Evidence,Date,URL
0,A video is being shared on social media with t...,False,A video shared on Facebook claims that a house...,06/04/2023,https://africacheck.org//fact-checks/meta-prog...
1,"Despite persistent rumours on social media, th...",False,A claim is circulating on Facebook in Nigeria ...,31/03/2023,https://africacheck.org//fact-checks/meta-prog...
2,The result of Nigeria's presidential election ...,False,Nigerians hate themselves but yet find it diff...,30/03/2023,https://africacheck.org//fact-checks/meta-prog...
3,An open letter addressed to Bola Tinubu is doi...,False,A post shared on Facebook claims that Adeyinka...,30/03/2023,https://africacheck.org//fact-checks/meta-prog...
4,As the courts examine Bola Tinubu’s electoral ...,False,Nigeria president-elect Bola Tinubu has said t...,28/03/2023,https://africacheck.org//fact-checks/meta-prog...
5,The election in this northeastern state in Nig...,/sites/default/files/inline-images/Binanichecked,Nigeria held elections for governors and state...,24/03/2023,https://africacheck.org//fact-checks/meta-prog...
6,A Nigerian-American professor of history winni...,/sites/default/files/inline-images/Saheefalse,Screenshots circulating on Facebook in Nigeria...,24/03/2023,https://africacheck.org//fact-checks/meta-prog...
7,Did a top Nigerian government official really ...,False,“I am now 100% against Binani becoming the Gov...,24/03/2023,https://africacheck.org//fact-checks/meta-prog...
8,"As Nigerians vote for governors, claims have b...",/sites/default/files/inline-images/Oworufalse,"A photo of Olumide Oworu, the Labour Party can...",18/03/2023,https://africacheck.org//fact-checks/meta-prog...
9,Just days before elections for governor across...,False,A message circulating on WhatsApp and Facebook...,17/03/2023,https://africacheck.org//fact-checks/meta-prog...


In [150]:
# save to csv file
df.to_csv("factcheckhub.csv", index=False)

In [13]:
claims_html

[]